In [11]:
import os
import glob
import time
import datetime
import collections
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snc

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib

import lightgbm
import PIL
import html5lib
import bs4

In [13]:
print(sklearn.__version__)
print(lightgbm.__version__)

0.21.3
2.2.3


In [14]:
%matplotlib inline

In [4]:
scores_df = pd.read_csv('../input/depression/data/scores.csv')

In [5]:
#scores_df

In [6]:
scores_df = scores_df.drop(['melanch', 'edu'], axis=1)

In [7]:
def get_data(path_to_data, scores_df):
    data_df = pd.read_csv('../input/depression/data/condition/condition_1.csv')

    for filename in scores_df['number'][1:]:
        if 'condition' in filename:
            data_df = pd.concat([data_df, pd.read_csv(path_to_data + filename + '.csv')], axis=0)
    data_df.reset_index(inplace=True)
    return data_df[data_df.columns.drop(['index'])]

In [8]:
#def get_data(path_to_data):
#   return pd.read_csv(path_to_data)

In [9]:
#def get_labels(path_to_labels):
#   return pd.read_csv(path_to_labels)

In [10]:
train_df = get_data('../input/depression/data/condition/', scores_df)

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551716 entries, 0 to 551715
Data columns (total 3 columns):
timestamp    551716 non-null object
date         551716 non-null object
activity     551716 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.6+ MB


In [12]:
#train_df.head()

In [13]:
#train_df.tail()

In [14]:
train_df.shape

(551716, 3)

In [15]:
#train_df.info()

In [16]:
le = LabelEncoder()

In [130]:
#clf_lg = LogisticRegression(solver='lbfgs' ,multi_class='multinomial', n_jobs=2)
clf_dt = DecisionTreeClassifier(min_impurity_decrease=0.001, max_depth=20)
clf_lgb1 = lightgbm.LGBMClassifier(objective='multiclass', max_depth=4, learning_rate=0.01, num_threads=12)

In [18]:
#train_X = pd.to_datetime(train_df[train_df.columns.drop(['timestamp', 'activity',])])
#train_X = (pd.to_datetime(train_df[train_df.columns.drop(['timestamp', 'activity'])])).dt.strftime('%Y-%m-%d  %H:%M%:%S').astype(np.float32)
#train_X = train_df[train_df.columns.drop(['timestamp', 'activity'])].split('-').astype(np.float32)
train_X = le.fit_transform(train_df[train_df.columns.drop(['date', 'activity'])])
#print(le.fit_transform(train_df[train_df.columns.drop(['date', 'activity'])]))
#train_X = 
#train_X_timestamp = pd.to_datetime(train_X.loc['date', :])
#train_X_timestamp.loc['timestamp', :] = pd.to_datetime(train_X.loc['timestamp', :])
#train_X_timestamp['timestamp'] =
#pd.to_datetime(train_X.loc['timestamp', :])
#temp_train_X = pd.DataFrame(index=[0:551715])
#temp_train_X = pd.DataFrame()
#max_idx = 0
'''
for i, date in enumerate(train_X['date']):
    #if i > max_idx:
    #    max_idx = i
    #splitted_timestamp = timestamp.split(' ')
    #train_X_timestamp.iloc[i, 'timestamp'] = np.dtype(splitted_timestamp[0], np.datetime_data)
    train_date = pd.to_datetime(date)
    train_X_timestamp.loc['date', :] = train_date
    
    #train_X_timestamp.iloc[i, 'timestamp'] = timestamp
'''
#print(max_idx)
train_y = train_df[train_df.columns.drop(['timestamp', 'date'])]

In [19]:
train_X[:10]

array([93298, 93299, 93300, 93301, 93302, 93303, 93304, 93305, 93306,
       93307])

In [20]:
#train_X[-10:]

In [21]:
train_X_reshaped = train_X.reshape(-1, 1)

In [22]:
train_X_reshaped.shape

(551716, 1)

In [34]:
start_time = time.time()
clf_dt.fit(train_X_reshaped, train_y)
#clf_gb.fit(train_X_reshaped, train_y)
print("total_time in min: {}".format((time.time() - start_time ) / 60))

total_time in min: 0.06583672364552816


In [35]:
clf_dt.get_depth()

6

In [28]:
start_time = time.time()
#clf_dt.fit(train_X_reshaped, train_y)
clf_lgb.fit(train_X_reshaped, train_y)
print("total_time in min: {}".format((time.time() - start_time ) / 60))

total_time in min: 13.583125547568004


In [29]:
clf_lgb.booster_.save_model('lgb_model_2019-09-19.txt')

In [30]:
joblib.dump(clf_lgb, 'clf_lgb.pkl')

['clf_lgb.pkl']

In [31]:
start_time = time.time()

for _ in range(0, 20):
    clf_lgb.predict(train_X_reshaped[:10000])
print("total prediction time in min: {}".format((time.time() - start_time ) / 60))

total prediction time in min: 3.405159135659536


In [10]:
#json_df = pd.read_json('https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/')
#pd.read_json?

In [21]:
#pd.read_json?

In [24]:
json_file = open('../input/real-data/1568884838599-steps.json')
json_str = json_file.read()
json_file.close()
print(json_str)

{
  "name": "1568884838599-steps.json",
  "bucket": "bicovery.appspot.com",
  "generation": "1568884834219655",
  "metageneration": "1",
  "contentType": "application/octet-stream",
  "timeCreated": "2019-09-19T09:20:34.219Z",
  "updated": "2019-09-19T09:20:34.219Z",
  "storageClass": "STANDARD",
  "size": "2",
  "md5Hash": "11FxOYiYfpMxmANj4kGJzg==",
  "contentEncoding": "identity",
  "contentDisposition": "inline; filename*=utf-8''1568884838599-steps.json",
  "crc32c": "dk29dg==",
  "etag": "CIft8rnH3OQCEAE=",
  "downloadTokens": "d85c0665-972b-423b-a9dc-07f208095668"
}


In [109]:
for line in json_str.splitlines():
    print(line)

{
  "name": "1568884838599-steps.json",
  "bucket": "bicovery.appspot.com",
  "generation": "1568884834219655",
  "metageneration": "1",
  "contentType": "application/octet-stream",
  "timeCreated": "2019-09-19T09:20:34.219Z",
  "updated": "2019-09-19T09:20:34.219Z",
  "storageClass": "STANDARD",
  "size": "2",
  "md5Hash": "11FxOYiYfpMxmANj4kGJzg==",
  "contentEncoding": "identity",
  "contentDisposition": "inline; filename*=utf-8''1568884838599-steps.json",
  "crc32c": "dk29dg==",
  "etag": "CIft8rnH3OQCEAE=",
  "downloadTokens": "d85c0665-972b-423b-a9dc-07f208095668"
}


In [41]:
json_str.splitlines?

In [29]:
#pd.read_json?

In [34]:
#json.dumps?

In [69]:
#json_df= pd.read_json('../input/real-data/1568884838599-steps.json', orient='table', compression=None)
#
#json_df= pd.read_json('../input/real-data/1568884838599-steps.json', orient='table', lines=True)
#json_df= pd.read_json('../input/real-data/1568884838599-steps.json',
                     # orient='split',
                     # dtype = {'column_name': object},
                      #lines=True
#                     )
#json_df= pd.read_json(json_str, lines=True)

In [74]:
#json.loads?
!pwd

/home/uldo/work/Archer/code


In [75]:
#!cp '../input/real-data/1568884839599-steps.json' .
#list_all_types

In [194]:
info_types = glob.glob('../input/real-data/*.json')

In [180]:
#info_types

{'../input/real-data/1568884829494-applications.json',
 '../input/real-data/1568884829506-keyboard.json',
 '../input/real-data/1568884835062-activities.json',
 '../input/real-data/1568884838599-steps.json',
 '../input/real-data/1568885429513-applications.json',
 '../input/real-data/1568885429520-keyboard.json',
 '../input/real-data/1568885439286-steps.json',
 '../input/real-data/1568885441657-activities.json',
 '../input/real-data/1568885735438-activities.json',
 '../input/real-data/1568886029530-applications.json',
 '../input/real-data/1568886029539-keyboard.json',
 '../input/real-data/1568886039649-steps.json',
 '../input/real-data/1568886094639-keyboard.json',
 '../input/real-data/1568886094645-applications.json',
 '../input/real-data/1568886096206-heart_bpm.json',
 '../input/real-data/1568886096339-activities.json',
 '../input/real-data/1568886096403-steps.json',
 '../input/real-data/1568886137722-keyboard.json',
 '../input/real-data/1568886137760-applications.json',
 '../input/rea

In [195]:
info_types = set([info_type[33:-5] for info_type in info_types])

In [196]:
info_types

{'activities',
 'applications',
 'heart_bpm',
 'keyboard',
 'location',
 'phone_calls',
 'steps'}

In [101]:
#json_dct = json.loads('1568884839599-activites.json')
f_dct3 = open('1568884835062-save-activities.json', 'r')
json_dct3 = json.load(f_dct3)
f_dct3.close()
print(json_dct3)

{'name': '1568884835062-activities.json', 'bucket': 'bicovery.appspot.com', 'generation': '1568884830714864', 'metageneration': '1', 'contentType': 'application/octet-stream', 'timeCreated': '2019-09-19T09:20:30.714Z', 'updated': '2019-09-19T09:20:30.714Z', 'storageClass': 'STANDARD', 'size': '2', 'md5Hash': '11FxOYiYfpMxmANj4kGJzg==', 'contentEncoding': 'identity', 'contentDisposition': "inline; filename*=utf-8''1568884835062-activities.json", 'crc32c': 'dk29dg==', 'etag': 'CPD3nLjH3OQCEAE=', 'downloadTokens': '09ab0d1a-7d0d-4091-a4a4-f1317a006801'}


In [107]:
#json_dct = json.loads('1568884839599-activites.json')
3#f_dct4 = open('1568884835062-save-activities.json', 'r')
#json_df4 = pd.read_json(f_dct4, index=[0])
#f_dct4.close()
#print(json_df4)

In [106]:
#pd.read_json?

In [108]:
#json_df = pd.read_json('1568884835062-save-activities.json')
#json_df = pd.DataFrame(data=json_dct1, index=[0])
#json_df = pd.read_json?

In [148]:
#json_df

In [102]:
#json_dict = {}
json_file = open('../input/real-data/1568884838599-steps.json')
json_dict = json.load(json_file)
json_file.close()
# for line in json_file.readline():
#     json_dict[line] = line
        
# json_file.close()
print(json_dict)

{'name': '1568884838599-steps.json', 'bucket': 'bicovery.appspot.com', 'generation': '1568884834219655', 'metageneration': '1', 'contentType': 'application/octet-stream', 'timeCreated': '2019-09-19T09:20:34.219Z', 'updated': '2019-09-19T09:20:34.219Z', 'storageClass': 'STANDARD', 'size': '2', 'md5Hash': '11FxOYiYfpMxmANj4kGJzg==', 'contentEncoding': 'identity', 'contentDisposition': "inline; filename*=utf-8''1568884838599-steps.json", 'crc32c': 'dk29dg==', 'etag': 'CIft8rnH3OQCEAE=', 'downloadTokens': 'd85c0665-972b-423b-a9dc-07f208095668'}


In [43]:
exp_json_file = open('exp_json_file.json', 'w')
#json.dumps(json_str)
#json.dump(json_str, 'exp_json_file.json')
json.dump(json_str, exp_json_file)
exp_json_file.flush()
exp_json_file.close()

In [47]:
#pd.read_json('exp_json_file.json')

In [149]:
# json_dict = {}
# for line in json_str.splitlines():
#     splitted_line = line.split(' ')
#     if splitted_line != ['{'] and splitted_line != ['}']:
#         print(splitted_line)
#         print(type(splitted_line))
        
#         json_dict[splitted_line[2]] = splitted_line[3]
# print(json_dict)
#
#json_dict = {json_line.split(' ')[0]: json_line.split(' ')[1]  for json_line in json_str.splitlines()}

In [133]:
# json_dict_prepared1 = {}
# for key, value in json_dict_prepared.items():
#     json_dict_prepared1[key[1:len(key) - 2]] = value[1:len(value) -2]
# print(json_dict_prepared1)

In [63]:
# f_js_dump= open('json_dict_prepared1_dump.json', 'w')
# json.dump(json_dict_prepared1, f_js_dump)
# f_js_dump.flush()
# f_js_dump.close()

In [88]:
# f_js_load = open('json_dict_prepared1_dump.json', 'r')
# json_dict_loaded = json.load(f_js_load)
# #f_js_load.flush()
# f_js_load.close()

In [134]:
#f_js_load

In [135]:
#json_dict_loaded

In [85]:
# f_js_load1 = open('json_dict_prepared1_dump.json', 'r')
# json_dict_loaded1 = json.load(f_js_load1)
# #f_js_load.flush()
# f_js_load1.close()

In [136]:
#print(json_dict_loaded1)

In [103]:
#json_dct = json.loads('json_dict_prepared1_dump.json')

In [126]:
def json_to_dataframe(json_files_list):
    json_file = open(json_files_list[0], 'r')
    json_dict = json.load(json_file)
    json_file.close()
    json_df = pd.DataFrame(json_dict, index=[0])
    for i, file_name in enumerate(json_files_list[1:]):
        json_file = open(file_name, 'r')
        json_dict = json.load(json_file)
        json_file.close()
        json_df_temp = pd.DataFrame(json_dict, index=[i + 1])
        json_df = pd.concat([json_df, json_df_temp])
    return json_df

In [111]:
files_list = os.listdir('../input/real-data/')

In [113]:
#files_list

In [141]:
activities_files_list_glob = glob.glob('../input/real-data/*-activities.json')

In [142]:
len(activities_files_list_glob)

70

In [143]:
activities_json_df = json_to_dataframe(activities_files_list_glob)

In [171]:
activities_json_df.head(10)

,name,bucket,generation,metageneration,contentType,timeCreated,updated,storageClass,size,md5Hash,contentEncoding,contentDisposition,crc32c,etag,downloadTokens
0,1568908500711-activities.json,bicovery.appspot.com,1568908496346121,1,application/octet-stream,2019-09-19T15:54:56.345Z,2019-09-19T15:54:56.345Z,STANDARD,5235,l/Lb+h21EFAWB1axkS8pGw==,identity,inline; filename*=utf-8''1568908500711-activit...,8VhQag==,CIm48Myf3eQCEAE=,1e78a6ff-59e2-412c-8621-afb3528b682e
1,1569249981774-activities.json,bicovery.appspot.com,1569249982796913,1,application/octet-stream,2019-09-23T14:46:22.796Z,2019-09-23T14:46:22.796Z,STANDARD,3361,UhuZ00o8+ows9EO39ILUTQ==,identity,inline; filename*=utf-8''1569249981774-activit...,/wrEKg==,CPGop96X5+QCEAE=,d5420cdd-86b6-4b4c-93ad-0b2080252c37
2,1568907846907-activities.json,bicovery.appspot.com,1568907842604680,1,application/octet-stream,2019-09-19T15:44:02.604Z,2019-09-19T15:44:02.604Z,STANDARD,5235,S92YWQl5/m6GaPSKcubEEA==,identity,inline; filename*=utf-8''1568907846907-activit...,z4j57w==,CIidk5Wd3eQCEAE=,7880888e-290e-426c-909e-81357102d8e5
3,1568994353910-activities.json,bicovery.appspot.com,1568994356288717,1,application/octet-stream,2019-09-20T15:45:56.288Z,2019-09-20T15:45:56.288Z,STANDARD,3512,Y1uRbaxojHqeuKpHzwo3qg==,identity,inline; filename*=utf-8''1568994353910-activit...,H9kYNw==,CM25i7rf3+QCEAE=,fbaf4c6b-27b7-49ef-9f2a-8abf9a844a69
4,1568884835062-activities.json,bicovery.appspot.com,1568884830714864,1,application/octet-stream,2019-09-19T09:20:30.714Z,2019-09-19T09:20:30.714Z,STANDARD,2,11FxOYiYfpMxmANj4kGJzg==,identity,inline; filename*=utf-8''1568884835062-activit...,dk29dg==,CPD3nLjH3OQCEAE=,09ab0d1a-7d0d-4091-a4a4-f1317a006801
5,1569247502859-activities.json,bicovery.appspot.com,1569247504297755,1,application/octet-stream,2019-09-23T14:05:04.297Z,2019-09-23T14:05:04.297Z,STANDARD,3361,RTQMWXtubrd4qS5+0Qvoog==,identity,inline; filename*=utf-8''1569247502859-activit...,JZQfBQ==,CJveu8CO5+QCEAE=,c7764f70-7da9-4cb4-8e64-3221330026c4
6,1568894833457-activities.json,bicovery.appspot.com,1568894829361208,1,application/octet-stream,2019-09-19T12:07:09.360Z,2019-09-19T12:07:09.360Z,STANDARD,5691,9BXL3LFmOgy6WzF5YBNb5g==,identity,inline; filename*=utf-8''1568894833457-activit...,5s9ryw==,CLjw+dfs3OQCEAE=,7855a95d-749e-4930-97f1-4deb309e6888
7,1569248061987-activities.json,bicovery.appspot.com,1569248063627630,1,application/octet-stream,2019-09-23T14:14:23.627Z,2019-09-23T14:14:23.627Z,STANDARD,3513,2ARP5nKGfIapTtTbpkRMjQ==,identity,inline; filename*=utf-8''1569248061987-activit...,WAiYlw==,CO7ClsuQ5+QCEAE=,85c046a5-2071-4d44-bc5c-82a7dab6b18c
8,1569247975795-activities.json,bicovery.appspot.com,1569247976889175,1,application/octet-stream,2019-09-23T14:12:56.888Z,2019-09-23T14:12:56.888Z,STANDARD,3361,DMCTfOYql3VWZb7K2DjjVQ==,identity,inline; filename*=utf-8''1569247975795-activit...,8spiEw==,CNe26KGQ5+QCEAE=,8930a51b-8746-4bc1-8b72-433c0917156b
9,1568895115583-activities.json,bicovery.appspot.com,1568895112607425,1,application/octet-stream,2019-09-19T12:11:52.607Z,2019-09-19T12:11:52.607Z,STANDARD,5691,iJnxEHL5Lsa6rK1GTHW1Xg==,identity,inline; filename*=utf-8''1568895115583-activit...,HvJ4Lw==,CMHtgd/t3OQCEAE=,338244ce-188c-475f-a560-848b0b64e8e9


In [145]:
#json_df =
activities_json_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 69
Data columns (total 15 columns):
name                  70 non-null object
bucket                70 non-null object
generation            70 non-null object
metageneration        70 non-null object
contentType           70 non-null object
timeCreated           70 non-null object
updated               70 non-null object
storageClass          70 non-null object
size                  70 non-null object
md5Hash               70 non-null object
contentEncoding       70 non-null object
contentDisposition    70 non-null object
crc32c                70 non-null object
etag                  70 non-null object
downloadTokens        70 non-null object
dtypes: object(15)
memory usage: 8.8+ KB


In [146]:
#json_df = le.fit_transform(json_df[json_df.columns.drop(['bucket', ''])])

In [132]:
#start_time = time.time()
#clf_dt.fit(train_X_reshaped, train_y)
#clf_lgb1.fit(json_df, np.zeros(len(json_df)))
#print("total_time in min: {}".format((time.time() - start_time ) / 60))

In [147]:
import requests

In [172]:
url ='https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/' + activities_json_df.iloc[9][0] + '?alt=media&token={}'.format(activities_json_df.iloc[9][14])
print(url)

https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1568895115583-activities.json?alt=media&token=338244ce-188c-475f-a560-848b0b64e8e9


In [209]:
firebase_url_part = 'https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/'

In [221]:
real_json = requests.get(url)

In [222]:
print(real_json)

<Response [200]>


In [223]:
print(real_json.json())

[{'dataType': 'com.google.activity.summary', 'endTime': 1568817884700, 'fields': {'duration': 9171697, 'activity': 3, 'num_segments': 1}, 'startTime': 1568808713003}, {'dataType': 'com.google.activity.summary', 'endTime': 1568818790927, 'fields': {'duration': 788934, 'activity': 0, 'num_segments': 1}, 'startTime': 1568818001993}, {'dataType': 'com.google.activity.summary', 'endTime': 1568819524072, 'fields': {'duration': 437339, 'activity': 0, 'num_segments': 1}, 'startTime': 1568819086733}, {'dataType': 'com.google.activity.summary', 'endTime': 1568841134161, 'fields': {'duration': 21610089, 'activity': 3, 'num_segments': 1}, 'startTime': 1568819524072}, {'dataType': 'com.google.activity.summary', 'endTime': 1568842920000, 'fields': {'duration': 1737276, 'activity': 3, 'num_segments': 1}, 'startTime': 1568841182724}, {'dataType': 'com.google.activity.summary', 'endTime': 1568844960000, 'fields': {'duration': 2040000, 'activity': 109, 'num_segments': 1}, 'startTime': 1568842920000}, {'

In [199]:
data_path = '../input/real-data/' 

In [232]:
requests_s=requests.session()
for info_type in info_types:
    files_info_type = glob.glob(data_path + '*-{}.json'.format(info_type))
    print("Number of " + info_type + " is " + str(len(files_info_type)) + "\n" * 3)
    for file_name in files_info_type:
        json_file = open(file_name)
        json_file_content = json.load(json_file)
        json_file.close()
        filename_url_part= file_name[19:]
        full_url = firebase_url_part + filename_url_part + '?alt=media&token={}'.format(json_file_content['downloadTokens'])
        print(full_url)  
        response = requests_s.get(full_url)
        print(response.text)
        print("\n"*3)
requests_s.close()

Number of applications is 77



https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1568890165103-applications.json?alt=media&token=33a6c3a6-2bd0-4bff-a6f5-14526624ee86
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569249979166-applications.json?alt=media&token=76d9be3a-020c-4e76-92e8-743685805d6b
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569247500802-applications.json?alt=media&token=a09052e7-e0ee-409b-9e2f-2f91ec6ec4cf
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1568895114215-applications.json?alt=media&token=73d5ae41-a4cf-4a16-85c6-6eb15ab37bb6
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1568984190553-applications.json?alt=media&token=3cf94a35-70d5-4817-8f32-148983044732
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569247032081-applications.json?alt=media&token=dff6c965-dcea-4773-bc8c-599beb16e705
[]
https://firebasestorage.googleapis.com

[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569247434117-applications.json?alt=media&token=6137d4ff-8aaf-4006-8405-76501d8b9cc5
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1568908416729-applications.json?alt=media&token=e1da66f0-39ff-49f2-9142-f9eab0be8fa4
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569249881451-applications.json?alt=media&token=15dd735a-fc82-49d1-98d2-821b65df2501
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569248061136-applications.json?alt=media&token=c9646b15-9a63-4d26-9971-c4759f3b21cc
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1568908171937-applications.json?alt=media&token=81f665fa-eea2-4ddf-bb6b-41fc740b540f
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569253772333-applications.json?alt=media&token=f8d744dc-9266-46b1-a5ca-f9a3c92a28a3
[]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/

[{"dataType":"com.google.activity.summary","endTime":1569165000000,"fields":{"duration":1420719,"activity":7,"num_segments":1},"startTime":1569163579281},{"dataType":"com.google.activity.summary","endTime":1569165660000,"fields":{"duration":660000,"activity":3,"num_segments":1},"startTime":1569165000000},{"dataType":"com.google.activity.summary","endTime":1569166800000,"fields":{"duration":1140000,"activity":7,"num_segments":1},"startTime":1569165660000},{"dataType":"com.google.activity.summary","endTime":1569169260000,"fields":{"duration":2340000,"activity":7,"num_segments":1},"startTime":1569166920000},{"dataType":"com.google.activity.summary","endTime":1569170040000,"fields":{"duration":480000,"activity":3,"num_segments":1},"startTime":1569169560000},{"dataType":"com.google.activity.summary","endTime":1569170880000,"fields":{"duration":840000,"activity":8,"num_segments":1},"startTime":1569170040000},{"dataType":"com.google.activity.summary","endTime":1569173220000,"fields":{"duratio

[{"dataType":"com.google.activity.summary","endTime":1568913329673,"fields":{"duration":5378081,"activity":3,"num_segments":1},"startTime":1568907951592},{"dataType":"com.google.activity.summary","endTime":1568918691719,"fields":{"duration":5221155,"activity":3,"num_segments":1},"startTime":1568913470564},{"dataType":"com.google.activity.summary","endTime":1568919134738,"fields":{"duration":306441,"activity":7,"num_segments":1},"startTime":1568918828297},{"dataType":"com.google.activity.summary","endTime":1568927872902,"fields":{"duration":8738164,"activity":3,"num_segments":1},"startTime":1568919134738},{"dataType":"com.google.activity.summary","endTime":1568955114573,"fields":{"duration":27199884,"activity":3,"num_segments":1},"startTime":1568927914689},{"dataType":"com.google.activity.summary","endTime":1568955975738,"fields":{"duration":819793,"activity":3,"num_segments":1},"startTime":1568955155945},{"dataType":"com.google.activity.summary","endTime":1568959301829,"fields":{"durat

[{"dataType":"com.google.activity.summary","endTime":1569163080000,"fields":{"duration":1418798,"activity":3,"num_segments":1},"startTime":1569161661202},{"dataType":"com.google.activity.summary","endTime":1569165000000,"fields":{"duration":1920000,"activity":7,"num_segments":1},"startTime":1569163080000},{"dataType":"com.google.activity.summary","endTime":1569165660000,"fields":{"duration":660000,"activity":3,"num_segments":1},"startTime":1569165000000},{"dataType":"com.google.activity.summary","endTime":1569166800000,"fields":{"duration":1140000,"activity":7,"num_segments":1},"startTime":1569165660000},{"dataType":"com.google.activity.summary","endTime":1569169260000,"fields":{"duration":2340000,"activity":7,"num_segments":1},"startTime":1569166920000},{"dataType":"com.google.activity.summary","endTime":1569170040000,"fields":{"duration":480000,"activity":3,"num_segments":1},"startTime":1569169560000},{"dataType":"com.google.activity.summary","endTime":1569170880000,"fields":{"durati

...

[{"dataType":"com.google.step_count.delta","endTime":1568841134161,"fields":{"steps":95},"startTime":1568830891747},{"dataType":"com.google.step_count.delta","endTime":1568842920000,"fields":{"steps":14},"startTime":1568841182724},{"dataType":"com.google.step_count.delta","endTime":1568844960000,"fields":{"steps":17},"startTime":1568842920000},{"dataType":"com.google.step_count.delta","endTime":1568845620000,"fields":{"steps":5},"startTime":1568844960000},{"dataType":"com.google.step_count.delta","endTime":1568846280000,"fields":{"steps":5},"startTime":1568845620000},{"dataType":"com.google.step_count.delta","endTime":1568847180000,"fields":{"steps":7},"startTime":1568846280000},{"dataType":"com.google.step_count.delta","endTime":1568847840000,"fields":{"steps":5},"startTime":1568847180000},{"dataType":"com.google.step_count.delta","endTime":1568849460000,"fields":{"steps":13},"startTime":1568847840000},{"dataType":"com.google.step_count.delta","endTime":1568850300000,"fields":{"steps"

[{"dataType":"com.google.step_count.delta","endTime":1569090918289,"fields":{"steps":68},"startTime":1569090612754},{"dataType":"com.google.step_count.delta","endTime":1569093107215,"fields":{"steps":31},"startTime":1569092720132},{"dataType":"com.google.step_count.delta","endTime":1569100465275,"fields":{"steps":38},"startTime":1569099225626},{"dataType":"com.google.step_count.delta","endTime":1569100465281,"fields":{"steps":0},"startTime":1569100465275}]
https://firebasestorage.googleapis.com/v0/b/bicovery.appspot.com/o/1569251252067-steps.json?alt=media&token=6336080b-2415-487e-b585-c614d5eeb40f
[{"dataType":"com.google.step_count.delta","endTime":1569165600000,"fields":{"steps":178},"startTime":1569165060000},{"dataType":"com.google.step_count.delta","endTime":1569166800000,"fields":{"steps":434},"startTime":1569165660000},{"dataType":"com.google.step_count.delta","endTime":1569169260000,"fields":{"steps":808},"startTime":1569166920000},{"dataType":"com.google.step_count.delta","en

[{"dataType":"com.google.step_count.delta","endTime":1568804100000,"fields":{"steps":30},"startTime":1568803764791},{"dataType":"com.google.step_count.delta","endTime":1568805240000,"fields":{"steps":127},"startTime":1568804460000},{"dataType":"com.google.step_count.delta","endTime":1568806920000,"fields":{"steps":336},"startTime":1568805300000},{"dataType":"com.google.step_count.delta","endTime":1568807580000,"fields":{"steps":0},"startTime":1568806980000},{"dataType":"com.google.step_count.delta","endTime":1568808540000,"fields":{"steps":140},"startTime":1568807640000},{"dataType":"com.google.step_count.delta","endTime":1568817884700,"fields":{"steps":73},"startTime":1568808600000},{"dataType":"com.google.step_count.delta","endTime":1568818790927,"fields":{"steps":53},"startTime":1568818001993},{"dataType":"com.google.step_count.delta","endTime":1568819524072,"fields":{"steps":79},"startTime":1568819086733},{"dataType":"com.google.step_count.delta","endTime":1568841134161,"fields":{"

[{"dataType":"com.google.step_count.delta","endTime":1569163020000,"fields":{"steps":47},"startTime":1569161586346},{"dataType":"com.google.step_count.delta","endTime":1569165000000,"fields":{"steps":1155},"startTime":1569163080000},{"dataType":"com.google.step_count.delta","endTime":1569165600000,"fields":{"steps":178},"startTime":1569165060000},{"dataType":"com.google.step_count.delta","endTime":1569166800000,"fields":{"steps":434},"startTime":1569165660000},{"dataType":"com.google.step_count.delta","endTime":1569169260000,"fields":{"steps":808},"startTime":1569166920000},{"dataType":"com.google.step_count.delta","endTime":1569169980000,"fields":{"steps":11},"startTime":1569169620000},{"dataType":"com.google.step_count.delta","endTime":1569170880000,"fields":{"steps":154},"startTime":1569170040000},{"dataType":"com.google.step_count.delta","endTime":1569173160000,"fields":{"steps":259},"startTime":1569170940000},{"dataType":"com.google.step_count.delta","endTime":1569173520000,"field

[{"dataType":"com.google.step_count.delta","endTime":1568804100000,"fields":{"steps":78},"startTime":1568803232185},{"dataType":"com.google.step_count.delta","endTime":1568805240000,"fields":{"steps":127},"startTime":1568804460000},{"dataType":"com.google.step_count.delta","endTime":1568806920000,"fields":{"steps":336},"startTime":1568805300000},{"dataType":"com.google.step_count.delta","endTime":1568807580000,"fields":{"steps":0},"startTime":1568806980000},{"dataType":"com.google.step_count.delta","endTime":1568808540000,"fields":{"steps":140},"startTime":1568807640000},{"dataType":"com.google.step_count.delta","endTime":1568817884700,"fields":{"steps":73},"startTime":1568808600000},{"dataType":"com.google.step_count.delta","endTime":1568818790927,"fields":{"steps":53},"startTime":1568818001993},{"dataType":"com.google.step_count.delta","endTime":1568819524072,"fields":{"steps":79},"startTime":1568819086733},{"dataType":"com.google.step_count.delta","endTime":1568841134161,"fields":{"